In [2]:
import rasterio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual

elevations = []
elev_raster = 'wc2.1_10m_elev.tif'
elev = (rasterio.open(elev_raster)).read()
rows = elev.shape[1]
cols = elev.shape[2]
elev = np.reshape(elev, (1, rows, cols))
elev = elev.astype("float")
elev[elev == -32768] = np.nan

m = 1195; l = 50
n = 285; k = 50

for i in range(int(n), int(n) + l):
     for j in range(int(m), int(m) + k):
            z = elev[0][i][j]
            elevations.append(z)
            
#Terrain
elevation = np.reshape(elevations, (l, k))
elevation[np.isnan(elevation)] = 0
data = pd.DataFrame(elevation)

# Transform it to a long format
df=data.unstack().reset_index()
df.columns=["X","Y","Z"]

# And transform the old column name in something numeric
df['X']=pd.Categorical(df['X'])
df['X']=df['X'].cat.codes

In [3]:
def terrain(a, e):
    fig = plt.figure(figsize = (8, 8))
    ax = fig.gca(projection='3d')
    ax.plot_trisurf(df['Y'], df['X'], df['Z'], cmap=plt.cm.viridis, linewidth=0.2)
    surf=ax.plot_trisurf(df['Y'], df['X'], df['Z'], cmap=plt.cm.viridis, linewidth=0.2)
    fig.colorbar(surf, shrink=0.5, aspect=5)
    ax.view_init(azim = a, elev = e)

interact(terrain, a = (0, 90, 5), e = (0, 90, 5));

interactive(children=(IntSlider(value=45, description='a', max=90, step=5), IntSlider(value=45, description='e…